In [12]:
%pip install uform

  Obtaining dependency information for uform from https://files.pythonhosted.org/packages/85/66/0b0f688e59e861b10d76231fd3fdedd68525a0b29b194cdce1500653b9d0/uform-1.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for transformers>=4.36.2 from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 596.1 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from PIL import Image
import torch
import glob
import regex as rg
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score


In [3]:
from helpers import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
device='mps'

In [6]:
ground_truth_de, ground_truth_en = create_ground_truth_dicts()

In [8]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large").to(device)


In [22]:
img_paths=glob.glob("data/Cliparts/01_Kate Hadfield/**/*.png", recursive=True)
imgs=pd.DataFrame({'path':img_paths})
imgs

,path
0,data/Cliparts/01_Kate Hadfield/logo-cropped.png
1,data/Cliparts/01_Kate Hadfield/EggHunting/khad...
2,data/Cliparts/01_Kate Hadfield/EggHunting/khad...
3,data/Cliparts/01_Kate Hadfield/EggHunting/khad...
4,data/Cliparts/01_Kate Hadfield/EggHunting/khad...
...,...
8212,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...
8213,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...
8214,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...
8215,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...


In [23]:
imgs['text']=imgs['path'].apply(lambda p:" ".join(rg.split(r'[\\|/_\\.-]', p)).strip())
imgs

,path,text
0,data/Cliparts/01_Kate Hadfield/logo-cropped.png,data Cliparts 01 Kate Hadfield logo cropped png
1,data/Cliparts/01_Kate Hadfield/EggHunting/khad...,data Cliparts 01 Kate Hadfield EggHunting khad...
2,data/Cliparts/01_Kate Hadfield/EggHunting/khad...,data Cliparts 01 Kate Hadfield EggHunting khad...
3,data/Cliparts/01_Kate Hadfield/EggHunting/khad...,data Cliparts 01 Kate Hadfield EggHunting khad...
4,data/Cliparts/01_Kate Hadfield/EggHunting/khad...,data Cliparts 01 Kate Hadfield EggHunting khad...
...,...,...
8212,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...,data Cliparts 01 Kate Hadfield DoctorsAndNurse...
8213,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...,data Cliparts 01 Kate Hadfield DoctorsAndNurse...
8214,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...,data Cliparts 01 Kate Hadfield DoctorsAndNurse...
8215,data/Cliparts/01_Kate Hadfield/DoctorsAndNurse...,data Cliparts 01 Kate Hadfield DoctorsAndNurse...


In [25]:
imgs["blip_text"] = ""
for i, img in tqdm(list(imgs.iterrows()), "encoding imgs"):
    img_dat = openImg(img["path"])
    if img_dat is not None:
        # img_dat=torch.Tensor(img_dat).to(device)
        inputs = processor(images=img_dat, return_tensors="pt").to(device)
        out = model.generate(max_new_tokens=128, **inputs)
        text = processor.decode(out[0], skip_special_tokens=True)
        imgs.loc[i, "blip_text"] = text

encoding imgs:   0%|          | 0/8217 [00:00<?, ?it/s]

cannot identify image file 'data/Cliparts/01_Kate Hadfield/GraduationStamps/BOLD/khadfield_GraduationStampsBOLD_star3.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/WeAllScreamForIceCream/WeAllScreamForIceCreamStamps/BOLD/khadfield_WASFIceCreamStamps_BOLD_waferstick1.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/FruitStand/khadfield_FruitStand_lemonslice2.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/MyKindaPet2/khadfield_MyKindaPet2_spider.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/AVeryGermanChristmas/khadfield_AVeryGermanChristmas_adventcalendar2.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/AVeryGermanChristmas/AVeryGermanChristmasStamps/DIGITAL STAMPS/khadfield_AVeryGermanChristmasStamps_threekingscake_2.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/WeekendAtHome/khadfield_WeekendAtHome_birdhouse.png'
cannot identify image file 'data/Cliparts/01_Kate Hadfield/SolarSystem/k

In [ ]:
# imgs.to_json('data/img_descriptions.json', orient='records')

In [ ]:
# imgs=pd.read_json('data/img_descriptions.json', orient='records')

In [28]:
imgs["full_text"] = imgs["text"] + " " + imgs["blip_text"]


def calculate_similarity_tfidf(query, k=20, col='full_text'):
    relevant_docs = []

    documents = imgs[col].to_list()

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

    query_tfidf = tfidf_vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    document_ranks = sorted(
        enumerate(cosine_similarities), key=lambda x: x[1], reverse=True
    )[:k]

    for rank, score in document_ranks:
        # print(f"{documents[rank]}: Similarity Score = {score:.4f}")
        relevant_docs.append(imgs.loc[rank, "text"])

    return relevant_docs


relevant_docs = calculate_similarity_tfidf("easter", 10)
print(relevant_docs)

['data Cliparts 01 Kate Hadfield CalendarYear khadfield CalendarYear Easterbasket png', 'data Cliparts 01 Kate Hadfield BigTop khadfield BigTop ball png', 'data Cliparts 01 Kate Hadfield BigTop BigTopStamps LINE ART khadfield BigTopStamps ball png', 'data Cliparts 01 Kate Hadfield BigTop BigTopStamps DIGITAL STAMPS khadfield BigTopStamps ball 2 png', 'data Cliparts 01 Kate Hadfield AnimalAntics AnimalAnticsStamps LINE ART khadfield AnimalAnticsStamps ball png', 'data Cliparts 01 Kate Hadfield AnimalAntics AnimalAnticsStamps DIGITAL STAMPS khadfield AnimalAnticsStamps ball 2 png', 'data Cliparts 01 Kate Hadfield CalendarYear khadfield CalendarYear easteregg png', 'data Cliparts 01 Kate Hadfield EggHunting khadfield EggHunting basket1 png', 'data Cliparts 01 Kate Hadfield Prehistoric PrehistoricStamps BOLD khadfield PrehistoricStamps BOLD egg1 png', 'data Cliparts 01 Kate Hadfield Prehistoric PrehistoricStamps LINE ART khadfield PrehistoricStamps egg1 png']


In [37]:
def precision_air(target, pred, k=20):
    rel_set = set(target)
    # print(rel_set)
    doc_set = set(pred[:k])
    tp = len(doc_set.intersection(rel_set))  # docs that are in both -relevant docs
    fp = len(
        doc_set.difference(rel_set)
    )  # docs that are not in relevant set - irrelevant docs (false positiv)
    fn = len(
        rel_set.difference(doc_set)
    )  # relevant docs that are not present in doc set - missing docs
    if tp == 0:
        return 0
    precision = tp / (tp + fp)
    return precision

In [38]:
# TF-IDF as baseline
def eval_query(
    q,
    ground_truth: dict[str, list[str]],
    col: str = "full_text",
    metric=precision_score,
):
    scores = []

    for query in ground_truth.keys():
        paths = ground_truth[query]

        tfidf_predictions = q(query, len(paths), col=col)
        f1_score_value = metric(paths, tfidf_predictions)
        scores.append(f1_score_value)
    return sum(scores) / len(scores)


print(
    f"Average precision score (TF-IDF+BLIP, EN): {eval_query(calculate_similarity_tfidf,ground_truth_en, metric=precision_air)}"
)
print(
    f"Average precision score (TF-IDF+BLIP, DE): {eval_query(calculate_similarity_tfidf,ground_truth_de, metric=precision_air)}"
)
print(
    f"Average precision score (TF-IDF, EN): {eval_query(calculate_similarity_tfidf,ground_truth_en, 'text', metric=precision_air)}"
)
print(
    f"Average precision score (TF-IDF, DE): {eval_query(calculate_similarity_tfidf,ground_truth_de, 'text', metric=precision_air)}"
)

Average precision score (TF-IDF+BLIP, EN): 0.4691910712744045
Average precision score (TF-IDF+BLIP, DE): 0.027777777777777776
Average precision score (TF-IDF, EN): 0.38690576398909726
Average precision score (TF-IDF, DE): 0.027777777777777776
